<a href="https://colab.research.google.com/github/khatriadbhut/Image_denoising_project/blob/main/Low_Light_enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

import keras
from keras import layers

import tensorflow as tf

In [2]:

!pip install gdown
import gdown

file_id = '1lNHqJmW8IJwpBbQkJC8ElfZZyBiWcDTi'
destination = 'lol_dataset.zip'

url = f"https://drive.google.com/uc?id={file_id}"

gdown.download(url, destination, quiet=False)

!unzip -q lol_dataset.zip && rm lol_dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1lNHqJmW8IJwpBbQkJC8ElfZZyBiWcDTi
From (redirected): https://drive.google.com/uc?id=1lNHqJmW8IJwpBbQkJC8ElfZZyBiWcDTi&confirm=t&uuid=c1982c57-0052-49a9-b06a-8e15405e8eb7
To: /content/lol_dataset.zip
100%|██████████| 347M/347M [00:13<00:00, 25.1MB/s]


In [3]:
SZ_IMG = 256
SZ_BATCH = 32
MAX_TRAIN_IMGS = 400

def create_dataset(image_list):
    ds = tf.data.Dataset.from_tensor_slices((image_list))
    ds = ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(SZ_BATCH, drop_remainder=True)
    return ds

def process_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(images=img, size=[SZ_IMG, SZ_IMG])
    img = img / 255.0
    return img

train_images = sorted(glob("./lol_dataset/our485/low/*"))[:MAX_TRAIN_IMGS]
val_images = sorted(glob("./lol_dataset/our485/low/*"))[MAX_TRAIN_IMGS:]
test_images = sorted(glob("./lol_dataset/eval15/low/*"))


train_ds = create_dataset(train_images)
val_ds = create_dataset(val_images)

print("Training Dataset:", train_ds)
print("Validation Dataset:", val_ds)

Training Dataset: <_BatchDataset element_spec=TensorSpec(shape=(32, 256, 256, 3), dtype=tf.float32, name=None)>
Validation Dataset: <_BatchDataset element_spec=TensorSpec(shape=(32, 256, 256, 3), dtype=tf.float32, name=None)>


In [4]:
def build_dce_net():
    input_image = keras.Input(shape=[None, None, 3])

    # First set of convolutional layers
    conv1 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(input_image)
    conv2 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv1)
    conv3 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv2)
    conv4 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv3)

    # Concatenation layers in between additional convolution layers
    concat_layer1 = layers.Concatenate(axis=-1)([conv4, conv3])
    conv5 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(concat_layer1)
    concat_layer2 = layers.Concatenate(axis=-1)([conv5, conv2])
    conv6 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(concat_layer2)
    concat_layer3 = layers.Concatenate(axis=-1)([conv6, conv1])

    # Output layer
    output_image = layers.Conv2D(24, (3, 3), strides=(1, 1), activation="tanh", padding="same")(concat_layer3)

    return keras.Model(inputs=input_image, outputs=output_image)


In [5]:
def color_constancy_loss(x):
    mean_rgb = tf.reduce_mean(x, axis=(1, 2), keepdims=True)
    mr, mg, mb = (
        mean_rgb[:, :, :, 0],
        mean_rgb[:, :, :, 1],
        mean_rgb[:, :, :, 2],
    )
    d_rg = tf.square(mr - mg)
    d_rb = tf.square(mr - mb)
    d_gb = tf.square(mb - mg)
    return tf.sqrt(tf.square(d_rg) + tf.square(d_rb) + tf.square(d_gb))


In [6]:

def exposure_loss(x, mean_val=0.6):
    x = tf.reduce_mean(x, axis=3, keepdims=True)
    mean = tf.nn.avg_pool2d(x, ksize=16, strides=16, padding="VALID")
    return tf.reduce_mean(tf.square(mean - mean_val))

In [7]:

def illumination_smoothness_loss(x):
    batch_size = tf.shape(x)[0]
    h_x = tf.shape(x)[1]
    w_x = tf.shape(x)[2]
    count_h = (tf.shape(x)[2] - 1) * tf.shape(x)[3]
    count_w = tf.shape(x)[2] * (tf.shape(x)[3] - 1)
    h_tv = tf.reduce_sum(tf.square((x[:, 1:, :, :] - x[:, : h_x - 1, :, :])))
    w_tv = tf.reduce_sum(tf.square((x[:, :, 1:, :] - x[:, :, : w_x - 1, :])))
    batch_size = tf.cast(batch_size, dtype=tf.float32)
    count_h = tf.cast(count_h, dtype=tf.float32)
    count_w = tf.cast(count_w, dtype=tf.float32)
    return 2 * (h_tv / count_h + w_tv / count_w) / batch_size


In [8]:

class SpatialConsistencyLoss(keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(reduction="none")

        self.left_kernel = tf.constant(
            [[[[0, 0, 0]], [[-1, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.right_kernel = tf.constant(
            [[[[0, 0, 0]], [[0, 1, -1]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.up_kernel = tf.constant(
            [[[[0, -1, 0]], [[0, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.down_kernel = tf.constant(
            [[[[0, 0, 0]], [[0, 1, 0]], [[0, -1, 0]]]], dtype=tf.float32
        )

    def call(self, y_true, y_pred):
        original_mean = tf.reduce_mean(y_true, 3, keepdims=True)
        enhanced_mean = tf.reduce_mean(y_pred, 3, keepdims=True)
        original_pool = tf.nn.avg_pool2d(
            original_mean, ksize=4, strides=4, padding="VALID"
        )
        enhanced_pool = tf.nn.avg_pool2d(
            enhanced_mean, ksize=4, strides=4, padding="VALID"
        )

        d_original_left = tf.nn.conv2d(
            original_pool,
            self.left_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        d_original_right = tf.nn.conv2d(
            original_pool,
            self.right_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        d_original_up = tf.nn.conv2d(
            original_pool, self.up_kernel, strides=[1, 1, 1, 1], padding="SAME"
        )
        d_original_down = tf.nn.conv2d(
            original_pool,
            self.down_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )

        d_enhanced_left = tf.nn.conv2d(
            enhanced_pool,
            self.left_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        d_enhanced_right = tf.nn.conv2d(
            enhanced_pool,
            self.right_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        d_enhanced_up = tf.nn.conv2d(
            enhanced_pool, self.up_kernel, strides=[1, 1, 1, 1], padding="SAME"
        )
        d_enhanced_down = tf.nn.conv2d(
            enhanced_pool,
            self.down_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )

        d_left = tf.square(d_original_left - d_enhanced_left)
        d_right = tf.square(d_original_right - d_enhanced_right)
        d_up = tf.square(d_original_up - d_enhanced_up)
        d_down = tf.square(d_original_down - d_enhanced_down)
        return d_left + d_right + d_up + d_down


In [9]:

class ZeroDCE(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dce_model = build_dce_net()

    def compile(self, learning_rate, **kwargs):
        super().compile(**kwargs)
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.spatial_constancy_loss = SpatialConsistencyLoss(reduction="none")
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.illumination_smoothness_loss_tracker = keras.metrics.Mean(
            name="illumination_smoothness_loss"
        )
        self.spatial_constancy_loss_tracker = keras.metrics.Mean(
            name="spatial_constancy_loss"
        )
        self.color_constancy_loss_tracker = keras.metrics.Mean(
            name="color_constancy_loss"
        )
        self.exposure_loss_tracker = keras.metrics.Mean(name="exposure_loss")
      #  self.perceptual_loss_tracker = keras.metrics.Mean(name="perceptual_loss")
        self.ssim_loss_tracker = keras.metrics.Mean(name="ssim_loss")
        self.total_variation_loss_tracker = keras.metrics.Mean(name="total_variation_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.illumination_smoothness_loss_tracker,
            self.spatial_constancy_loss_tracker,
            self.color_constancy_loss_tracker,
            self.exposure_loss_tracker,
     #       self.perceptual_loss_tracker,
            self.ssim_loss_tracker,
            self.total_variation_loss_tracker,
        ]

    def get_enhanced_image(self, data, output):
        r1 = output[:, :, :, :3]
        r2 = output[:, :, :, 3:6]
        r3 = output[:, :, :, 6:9]
        r4 = output[:, :, :, 9:12]
        r5 = output[:, :, :, 12:15]
        r6 = output[:, :, :, 15:18]
        r7 = output[:, :, :, 18:21]
        r8 = output[:, :, :, 21:24]
        x = data + r1 * (tf.square(data) - data)
        x = x + r2 * (tf.square(x) - x)
        x = x + r3 * (tf.square(x) - x)
        enhanced_image = x + r4 * (tf.square(x) - x)
        x = enhanced_image + r5 * (tf.square(enhanced_image) - enhanced_image)
        x = x + r6 * (tf.square(x) - x)
        x = x + r7 * (tf.square(x) - x)
        enhanced_image = x + r8 * (tf.square(x) - x)
        return enhanced_image

    def call(self, data):
        dce_net_output = self.dce_model(data)
        return self.get_enhanced_image(data, dce_net_output)

    def compute_losses(self, data, output):
        enhanced_image = self.get_enhanced_image(data, output)
        loss_illumination = 200 * illumination_smoothness_loss(output)
        loss_spatial_constancy = tf.reduce_mean(
            self.spatial_constancy_loss(enhanced_image, data)
        )
        loss_color_constancy = 5 * tf.reduce_mean(color_constancy_loss(enhanced_image))
        loss_exposure = 10 * tf.reduce_mean(exposure_loss(enhanced_image))

    #    perceptual_loss = VGGPerceptualLoss()(data, enhanced_image)
        ssim_loss = 1 - tf.image.ssim(data, enhanced_image, max_val=1.0)
        tv_loss = tf.reduce_sum(tf.image.total_variation(enhanced_image))

        total_loss = (
            loss_illumination
            + loss_spatial_constancy
            + loss_color_constancy
            + loss_exposure
   #         + perceptual_loss
            + ssim_loss
            + 0.1 * tv_loss
        )

        return {
            "total_loss": total_loss,
            "illumination_smoothness_loss": loss_illumination,
            "spatial_constancy_loss": loss_spatial_constancy,
            "color_constancy_loss": loss_color_constancy,
            "exposure_loss": loss_exposure,
  #          "perceptual_loss": perceptual_loss,
            "ssim_loss": ssim_loss,
            "total_variation_loss": tv_loss,
        }

    def train_step(self, data):
        with tf.GradientTape() as tape:
            output = self.dce_model(data)
            losses = self.compute_losses(data, output)

        gradients = tape.gradient(
            losses["total_loss"], self.dce_model.trainable_weights
        )
        self.optimizer.apply_gradients(zip(gradients, self.dce_model.trainable_weights))

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.illumination_smoothness_loss_tracker.update_state(
            losses["illumination_smoothness_loss"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.color_constancy_loss_tracker.update_state(losses["color_constancy_loss"])
        self.exposure_loss_tracker.update_state(losses["exposure_loss"])
 #       self.perceptual_loss_tracker.update_state(losses["perceptual_loss"])
        self.ssim_loss_tracker.update_state(losses["ssim_loss"])
        self.total_variation_loss_tracker.update_state(losses["total_variation_loss"])

        return {metric.name: metric.result() for metric in self.metrics}

    def test_step(self, data):
        output = self.dce_model(data)
        losses = self.compute_losses(data, output)

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.illumination_smoothness_loss_tracker.update_state(
            losses["illumination_smoothness_loss"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.color_constancy_loss_tracker.update_state(losses["color_constancy_loss"])
        self.exposure_loss_tracker.update_state(losses["exposure_loss"])
#        self.perceptual_loss_tracker.update_state(losses["perceptual_loss"])
        self.ssim_loss_tracker.update_state(losses["ssim_loss"])
        self.total_variation_loss_tracker.update_state(losses["total_variation_loss"])

        return {metric.name: metric.result() for metric in self.metrics}

    def save_weights(self, filepath, overwrite=True, save_format=None, options=None):
        """While saving the weights, we simply save the weights of the DCE-Net"""
        self.dce_model.save_weights(
            filepath,
            overwrite=overwrite,
            save_format=save_format,
            options=options,
        )

    def load_weights(self, filepath, by_name=False, skip_mismatch=False, options=None):
        """While loading the weights, we simply load the weights of the DCE-Net"""
        self.dce_model.load_weights(
            filepath=filepath,
            by_name=by_name,
            skip_mismatch=skip_mismatch,
            options=options,
        )


In [ ]:
# Initialize and compile the model
zero_dce_model = ZeroDCE()
zero_dce_model.compile(learning_rate=1e-4)

# Train the model
history = zero_dce_model.fit(train_ds, validation_data=val_ds, epochs=100)

# Function to plot the training and validation losses over epochs
def plot_result(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()

# Plot the losses
plot_result("total_loss")
plot_result("illumination_smoothness_loss")
plot_result("spatial_constancy_loss")
plot_result("color_constancy_loss")
plot_result("exposure_loss")
#plot_result("perceptual_loss")
plot_result("ssim_loss")
plot_result("total_variation_loss")


Epoch 1/100
12/12 [==============================] - 935s 76s/step - total_loss: 8725.1162 - illumination_smoothness_loss: 4.0346 - spatial_constancy_loss: 5.5981e-05 - color_constancy_loss: 0.0028 - exposure_loss: 2.9788 - ssim_loss: 0.0019 - total_variation_loss: 87180.9766 - val_total_loss: 11788.4854 - val_illumination_smoothness_loss: 7.1653 - val_spatial_constancy_loss: 1.6431e-04 - val_color_constancy_loss: 0.0023 - val_exposure_loss: 3.0053 - val_ssim_loss: 0.0061 - val_total_variation_loss: 117783.0625
Epoch 2/100
12/12 [==============================] - 944s 79s/step - total_loss: 7642.2017 - illumination_smoothness_loss: 4.6672 - spatial_constancy_loss: 8.6563e-04 - color_constancy_loss: 0.0022 - exposure_loss: 3.0622 - ssim_loss: 0.0186 - total_variation_loss: 76344.5078 - val_total_loss: 9674.4355 - val_illumination_smoothness_loss: 8.7238 - val_spatial_constancy_loss: 0.0013 - val_color_constancy_loss: 0.0015 - val_exposure_loss: 3.1212 - val_ssim_loss: 0.0504 - val_total

In [ ]:

def plot_results(images, titles, figure_size=(12, 12)):
    fig = plt.figure(figsize=figure_size)
    for i in range(len(images)):
        fig.add_subplot(1, len(images), i + 1).set_title(titles[i])
        _ = plt.imshow(images[i])
        plt.axis("off")
    plt.show()


def infer(original_image):
    image = keras.utils.img_to_array(original_image)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    output_image = zero_dce_model(image)
    output_image = tf.cast((output_image[0, :, :, :] * 255), dtype=np.uint8)
    output_image = Image.fromarray(output_image.numpy())
    return output_image

In [ ]:
for val_image_file in test_images:
    original_image = Image.open(val_image_file)
    enhanced_image = infer(original_image)
    plot_results(
        [original_image, ImageOps.autocontrast(original_image), enhanced_image],
        ["Original", "PIL Autocontrast", "Enhanced"],
        (20, 12),
    )

In [ ]:
def calculate_psnr_for_model(model, test_dataset):

    psnr_values = []

    for data in test_dataset:
        true_images = data
        predicted_images = model(data, training=False)  # Get model predictions
        psnr_value = tf.image.psnr(true_images, predicted_images, max_val=1.0)
        psnr_values.append(psnr_value)

    average_psnr = tf.reduce_mean(psnr_values)
    return average_psnr.numpy()

average_psnr = calculate_psnr_for_model(zero_dce_model, test_ds)
print(f"Average PSNR for the test dataset: {average_psnr:.2f} dB")
